#Image analysis in research paper



#### Install libraries needed

In [ ]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core  faiss-cpu  pypdf langchain_community mdutils datasets langchain_huggingface xmltodict sentence-transformers langchain_groq langchain_ollama langchain_mistralai fitz jq langchain_cohere frontend
%pip install -U PyMuPDF openai rapidocr-onnxruntime
%pip install --upgrade --quiet "unstructured[all-docs]"

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 10s (491 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

### Access data in the google drive from colab notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Imports

In [ ]:
from PIL import Image
import io
import os
from dotenv import load_dotenv

import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
#from langchain_cohere import ChatCohere, CohereEmbeddings
# Load environment variables from a .env file
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')

## Langsmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["TAVILY_API_KEY"]=userdata.get('TAVILY_API_KEY')
os.environ["COHERE_API_KEY"]=userdata.get('COHERE_API_KEY')
import nest_asyncio
from typing import List, TypedDict, Any, Tuple
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END, START
from datetime import date, datetime
from langgraph.checkpoint.memory import MemorySaver
import getpass
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_cohere import ChatCohere, CohereEmbeddings
import fitz  # PyMuPDF
from PIL import Image
import io
import os
import base64
import httpx
from langchain_core.messages import HumanMessage
import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_cohere import ChatCohere, CohereEmbeddings
nest_asyncio.apply()
from google.colab import drive
from langchain_core.pydantic_v1 import BaseModel, Field
from pprint import pprint
import functools
import operator
from typing import Sequence
from typing_extensions import TypedDict
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import PromptTemplate
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


### Extract text data and extract images into a folder

1. Loop through the pages, extract the textual data.
2. Extract the images in the PDF and save them with the extention in the specified page


In [ ]:
def save_images(filePath):
  text_data = []
  img_data = []
  with fitz.open(filePath) as pdf_file:
    # Create a directory to store the images
    if not os.path.exists("extracted_images"):
        os.makedirs("extracted_images")

    # Loop through every page in the PDF
    for page_number in range(len(pdf_file)):
        page = pdf_file[page_number]

        # Get the text on page
        text = page.get_text().strip()
        text_data.append({"response": text, "name": page_number+1})
        # Get the list of images on the page
        images = page.get_images(full=True)

        # Loop through all images found on the page
        for image_index, img in enumerate(images, start=0):
            xref = img[0]  # Get the XREF of the image
            base_image = pdf_file.extract_image(xref)  # Extract the image
            image_bytes = base_image["image"]  # Get the image bytes
            image_ext = base_image["ext"]  # Get the image extension

            # Load the image using PIL and save it
            image = Image.open(io.BytesIO(image_bytes))
            image.save(f"extracted_images/image_{page_number+1}_{image_index+1}.{image_ext}")
  return text_data, img_data

In [ ]:
text_data, img_data=save_images("/content/drive/MyDrive/ColabNotebooks/colab_data/ResearchPubAI/testData/S7.pdf")

Contents of each page stored in response and name as page number

In [ ]:
text_data

[{'response': 'Nanophotonics – poised to revolutionize displays and \nimaging!\n1\nSource: DARPA SWEEPER program 2015\nLiDAR\n3D displays\nVirtual reality\nArtificial vision\nOptical computing',
  'name': 1},
 {'response': 'Quantum materials exhibit fast and large tunability!\n2\nA new class of tunable optical materials: Quantum materials! \nEither small or slow tuning speed limit the application\n• Mechanical and thermal tuning can be slow\n• Small tunability due to a single mechanism involved in light-matter interaction\nh\nυ\nfs\n~\np\ns\nele\nctr\non\nConventional Materials: \nSingle particle response\nhυ\nns~ms\nQuantum Materials:\nCollective response\nP. Gopalan, B. Sensale-Rodriguez, Adv. Opt. Mater. 2019.      K. Hallman, et al., Adv. Opt. Mater. (2020).',
  'name': 2},
 {'response': '1T-TaS2 exhibits charge density waves at room \ntemperature\n3\nJournal of Electron Spectroscopy and Related Phenomena, \nStrongly correlated systems, 117–118 (2001): 433–49. \n \nCharge density w

#### Image data yet to be processed

In [ ]:
img_data

[]

### Analysed how unstructured works.

In [ ]:
from langchain_community.document_loaders.image import UnstructuredImageLoader
#/content/drive/MyDrive/Colab Notebooks/colab_data/extracted_images/image_7_3.jpeg
# loads the text in the images
loader = UnstructuredImageLoader("/content/drive/MyDrive/ColabNotebooks/colab_data/extracted_images/image_7_3.jpeg")
data = loader.load()
data[0]

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

Document(metadata={'source': '/content/drive/MyDrive/ColabNotebooks/colab_data/extracted_images/image_7_3.jpeg'}, page_content='(a)\n\n—_Interlayer\n\nPhoto energy\n\na\n\n{TTaSs y\n\nthin film\n\n(b) Substrate\n\nwhite light\n\nroom T')

## Relevant images are filtered

Step 1: Create a data model, A binary score is used to access the relevance of image with respect to contents of the paper.

Step 2: For each image check the relevance and remove unnecessary images that is not relevant to the context of the paper. For example logos, names and other trivial data. Done using gpt-40 model as it gives great results for image analysis.

Step 3: Collect all the relevance metrics and keep those images for further processing.  

In [ ]:
# Data model
class AnalyseRelevance(BaseModel):
    """Binary score to assess the relevance of image with respect to the contents of the paper."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )
relevant_images=[]

def checkRelevance():
  for img in os.listdir("/content/drive/MyDrive/ColabNotebooks/colab_data/extracted_images/"):
    print(img)
    im = "extracted_images/{img}"
    with open(os.path.join("/content/drive/MyDrive/ColabNotebooks/colab_data/extracted_images", img), "rb") as image_file:
      image_bytes = image_file.read()

    image_data = base64.b64encode(image_bytes).decode("utf-8") # encoding image bytes to base64
    prompt="You are an assistant tasked to analyse images and indicate whether the image is relevant or not with the contents of the slide.\
            Images that contain logos, University names are considered as trivial to the slide paper data. Henceforth it is classified as 'no'. \
            An image that does not contain anything is classified as a 'no' \
            Everything else is classified as an 'yes'\
            Give a binary score 'yes' or 'no'. Yes' means that the image is important to the data in the slide."
    message = HumanMessage(
    content=[
        {"type": "text", "text": prompt},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
      ]
    )
    llm = ChatOpenAI(model="gpt-4o")
    structured_llm_grader = llm.with_structured_output(AnalyseRelevance)

    ai_msg = structured_llm_grader.invoke([message])
    print(ai_msg.binary_score)
    if ai_msg.binary_score == "yes":
      relevant_images.append(img)
      print(img)
      print("Is relevant!")
  return relevant_images

Analyse and describe each image in the form of a paragraph using gpt-4o

In [ ]:
def generate_image_details():
  for img in os.listdir("/content/drive/MyDrive/ColabNotebooks/colab_data/extracted_images/"):
    if img in relevant_images:
      print(img)
      im = "extracted_images/{img}"
      with open(os.path.join("/content/drive/MyDrive/ColabNotebooks/colab_data/extracted_images", img), "rb") as image_file:
        image_bytes = image_file.read()

      image_data = base64.b64encode(image_bytes).decode("utf-8") # encoding image bytes to base64
      prompt="You are an assistant tasked to analyse images. Present a paragraph describing the image."
      message = HumanMessage(
      content=[
          {"type": "text", "text": prompt},
          {
              "type": "image_url",
              "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
          },
        ]
      )
      llm = ChatOpenAI(model="gpt-4o")
      ai_msg = llm.invoke([message])
      print(ai_msg.content)
      img_data.append({"response":ai_msg.content, "name": img})
  return img_data

In [ ]:
checkRelevance()

image_4_1.jpeg
no
image_1_1.jpeg
no
image_1_2.jpeg
yes
image_1_2.jpeg
Is relevant!
image_3_3.jpeg
no
image_1_6.jpeg
yes
image_1_6.jpeg
Is relevant!
image_1_5.jpeg
yes
image_1_5.jpeg
Is relevant!
image_3_4.jpeg
yes
image_3_4.jpeg
Is relevant!
image_2_3.jpeg
yes
image_2_3.jpeg
Is relevant!
image_3_1.jpeg
no
image_1_4.jpeg
yes
image_1_4.jpeg
Is relevant!
image_2_1.jpeg
no
image_2_2.jpeg
yes
image_2_2.jpeg
Is relevant!
image_1_3.jpeg
yes
image_1_3.jpeg
Is relevant!
image_4_3.jpeg
yes
image_4_3.jpeg
Is relevant!
image_3_2.jpeg
yes
image_3_2.jpeg
Is relevant!
image_4_4.jpeg
yes
image_4_4.jpeg
Is relevant!
image_4_2.jpeg
yes
image_4_2.jpeg
Is relevant!
image_5_4.jpeg
yes
image_5_4.jpeg
Is relevant!
image_10_4.jpeg
yes
image_10_4.jpeg
Is relevant!
image_7_3.jpeg
yes
image_7_3.jpeg
Is relevant!
image_9_2.jpeg
yes
image_9_2.jpeg
Is relevant!
image_5_5.jpeg
yes
image_5_5.jpeg
Is relevant!
image_5_6.jpeg
no
image_7_1.jpeg
no
image_6_7.jpeg
yes
image_6_7.jpeg
Is relevant!
image_8_2.jpeg
yes
image_8

['image_1_2.jpeg',
 'image_1_6.jpeg',
 'image_1_5.jpeg',
 'image_3_4.jpeg',
 'image_2_3.jpeg',
 'image_1_4.jpeg',
 'image_2_2.jpeg',
 'image_1_3.jpeg',
 'image_4_3.jpeg',
 'image_3_2.jpeg',
 'image_4_4.jpeg',
 'image_4_2.jpeg',
 'image_5_4.jpeg',
 'image_10_4.jpeg',
 'image_7_3.jpeg',
 'image_9_2.jpeg',
 'image_5_5.jpeg',
 'image_6_7.jpeg',
 'image_8_2.jpeg',
 'image_10_13.jpeg',
 'image_10_2.jpeg',
 'image_6_4.jpeg',
 'image_10_3.jpeg',
 'image_9_3.jpeg',
 'image_10_12.jpeg',
 'image_6_2.jpeg',
 'image_6_3.jpeg',
 'image_10_14.jpeg',
 'image_5_8.jpeg',
 'image_8_3.jpeg',
 'image_5_2.jpeg',
 'image_6_5.jpeg',
 'image_11_2.jpeg',
 'image_5_3.jpeg']

In [ ]:
len(relevant_images)
sorted(relevant_images)

['image_10_12.jpeg',
 'image_10_13.jpeg',
 'image_10_14.jpeg',
 'image_10_2.jpeg',
 'image_10_3.jpeg',
 'image_10_4.jpeg',
 'image_11_2.jpeg',
 'image_1_2.jpeg',
 'image_1_3.jpeg',
 'image_1_4.jpeg',
 'image_1_5.jpeg',
 'image_1_6.jpeg',
 'image_2_2.jpeg',
 'image_2_3.jpeg',
 'image_3_2.jpeg',
 'image_3_4.jpeg',
 'image_4_2.jpeg',
 'image_4_3.jpeg',
 'image_4_4.jpeg',
 'image_5_2.jpeg',
 'image_5_3.jpeg',
 'image_5_4.jpeg',
 'image_5_5.jpeg',
 'image_5_8.jpeg',
 'image_6_2.jpeg',
 'image_6_3.jpeg',
 'image_6_4.jpeg',
 'image_6_5.jpeg',
 'image_6_7.jpeg',
 'image_7_3.jpeg',
 'image_8_2.jpeg',
 'image_8_3.jpeg',
 'image_9_2.jpeg',
 'image_9_3.jpeg']

Using the relevant image list, Call the image analysis function and storing the responses in JSON format.

In [ ]:
img_data=generate_image_details()
img_data

image_1_2.jpeg
The image is an artist's concept depicting advanced military technology with a focus on sensor integration. A soldier is equipped with wearable micro-sensors, allowing for real-time data access. Nearby, a military vehicle is shown with multiple small, low-power sensors attached, providing comprehensive environmental awareness. In the foreground, a compact unmanned aerial vehicle (UAV) features long-range sensors, contributing to the situational data available to the personnel. The scene is overlaid with colored lines representing sensor data fields, illustrating the seamless flow of information between different technological components.
image_1_6.jpeg
The image depicts a close-up view of a small electronic component being held by tweezers near a circuit board. The circuit board features various pathways, solder points, and components, indicating a detailed design typical of modern electronics. The electronic component in the tweezers appears to be delicate and possibly 

[{'response': "The image is an artist's concept illustrating advanced military technology integration on the battlefield. It depicts a soldier equipped with wearable micro-sensors, including a device on a helmet and what appears to be a smartwatch, suggesting that all sensor data is readily available to the warfighter. Nearby, a military vehicle is fitted with multiple small, low-power sensors. Additionally, a compact unmanned aerial vehicle (UAV) with long-range sensors is shown, indicating its role in surveillance and data collection. The environment is enhanced with colorful, layered lines representing sensor fields and data networks, highlighting the interconnectedness of these technologies in a tactical scenario.",
  'name': 'image_1_2.jpeg'},
 {'response': 'The image depicts a close-up view of a circuit board and a small electronic component held by tweezers. The circuit board features visible traces, solder points, and what appears to be a connector or socket. The component held

In [ ]:
len(img_data)

63

As you can see the total images of the data are 63 and only 29 are relevant to the context.

In [ ]:
len(relevant_images)

29